### Content:
- Determine Machine
- [Import Blocks](#import)
- [Orient on Machining Table](#orient)
- [Generate Blanks](#blank)
- [Generate Wire Path](#wires)


### Exercise:
- Ex. 9.1  Orient Block for Cutting
- Ex. 9.2  Add Geometry of Cutting Material
- Ex. 9.3  Place Block on Machine Bed
- Ex. 9.4  Generate Blank Material
- Ex. 9.5  Generate Wire Cutter Path & Output
</br>
---

<a id='import'></a>
<img style="float: right;" src="img/diagrams/1_allBlocks.png" width="400">
## Step 1: Import the blocks and turn it into a usable format
Locate the `.json` file with the exported discretized blocks.


In [1]:
import os
import compas

from compas.datastructures  import Mesh
from compas_view2.app import App

# 1. load from disk
blocks = [block for block in compas.json_load("07_blocks_flat_top_wirecutting.json")]

# 2. visualise the blocks
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

for block in blocks:
    viewer.add(block)
viewer.show()

<img style="float: right;" src="img/diagrams/2_singleBlock.png" width="400">

## Step 2: Isolate one Block

This notebook will go through the entire workflow on just one block. We will isolate an arbitrary block to find the wirecutting paths for.



In [2]:
import os
import compas

from compas.datastructures  import Mesh
from compas_view2.app import App

# 1. select block from list blocks
my_block = blocks[0]

# 2. visualise individual block
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(my_block)
viewer.show()

<img style="float: right;" src="img/diagrams/3_singleBlockColor.png" width="400">

## Step 3: Identify the top and bottom faces of the block

Using the face normals, we compare the face normal to the Z-axis to find the most vertical outside face. We then color-code the faces to ensure we are selecting the correct ones.

Ideally the `top` and `bottom` information would be embedded in the mesh, however in this case we are coding this to be independent of that data.


In [3]:
import os
import compas

from compas.datastructures  import Mesh
from compas.geometry  import Vector
from compas_view2.app import App

faces = list(my_block.faces())

# 1. find top face
top = sorted(my_block.faces(), key=lambda face: Vector(* my_block.face_normal(face, unitized=True)).dot([0,0,1]))[0]
my_block.face_attribute(top, 'top', True)

bottom = sorted(my_block.faces(), key=lambda face: Vector(* my_block.face_normal(face, unitized=True)).dot([0,0,1]))[-1]
my_block.face_attribute(bottom, 'bottom', True)

# 5. visualise individual block with top face different color
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(my_block, facecolors={top: (255,0,0), bottom: (0,128,0)}, opacity=0.7)
viewer.show()

<a id='orient'></a>
<img style="float: right;" src="img/diagrams/4_roorientBlock.png" width="400">

## Step 4: Orient the Block on the WorldXY plane

This step orients the block to be within the machine space and placed on the `worldXY` plane. Visualising the machine space ensures that we are cutting a block which is sized appropriately to the machinery.



In [4]:
import os
import compas

from compas.datastructures  import Mesh
from compas.geometry import bestfit_frame_numpy
from compas.geometry  import Frame, Rotation, Transformation, Plane
from compas.geometry  import Box
from compas_view2.app import App

# 1. get a list of the top faces
top = list(my_block.faces_where({'top': True}))[0]

# 2. get corner vertex coordinates the top faces
corners = my_block.face_coordinates(top)

# 3. generate a bestfit frame of the corners
frame = Frame(*bestfit_frame_numpy(corners))

# 4. generate a world frame
world = Frame.worldXY()

# 5. build the frame to frame transformation
X = Transformation.from_frame_to_frame(frame, world)

# 6. perform the transformation on the block
transformed_block = my_block.transformed(X)

# 7. flip the block to make it oriented for the wirecutting
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]

X2 = Rotation.from_axis_and_angle(yaxis, 3.14159)
rotated_block = transformed_block.transformed(X2)

# 7. set new variable name for the block
final_block = rotated_block


# 9. generate the machining workspace at worldXY
machine_dim = [24.00, 12.00, 15.50]
machine_space = Box(world,machine_dim[0],machine_dim[1],machine_dim[2])


# 8. visualize the correctly oriented block within the machining workspace
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(my_block, facecolors={top: (255,0,0), bottom: (0,128,0)}, opacity=0.5)
viewer.add(final_block, facecolors={top: (255,0,0), bottom: (0,128,0)})

viewer.add(machine_space, opacity=0.2)

viewer.show()

<a id='blank'></a>
<img style="float: right;" src="img/diagrams/5_blank.png" width="400">

## Step 5: Generate the blank material
This step is done very simply, and finds a dimension of blank material which fully encompasses the block with some buffer space. Ideally further steps would be taken to confirm that this is compatible with the actual blank material dimensions, preferably generating a blank with those dimensions as well.



In [5]:
import os
import compas

from compas.geometry import Frame, Box
from compas.geometry import Scale
from compas.datastructures import Mesh
from compas.geometry import oriented_bounding_box_numpy
from compas_view2.app import App


# 1. compute the bounding box of the block mesh
#    and convert it into a box geometry object

bbox = final_block.vertices_attributes('xyz', keys=final_block.vertices())
box = oriented_bounding_box_numpy(bbox)
blank = Box.from_bounding_box(box)
blank_unsized = Box.from_bounding_box(box)
bbf = blank.frame

# 2. add padding to blank material Box object by scaling up in every direction
#    use the frame of the blank as the origin for scaling up

blank.transform(Scale.from_factors([1.10, 1.10, 1.10], frame=bbf))


# 3. visualize the correctly oriented block within the machining workspace
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(blank, opacity=0.5)
viewer.add(final_block)
viewer.show()


<img style="float: right;" src="img/diagrams/6_edge.png" width="400">

## Step 6: Find a side edge

This step ensures that we are selecting an edge which is used in the next step to find the strip of edges located on the inclined sides of the block.



In [6]:
import os
import compas

from compas.geometry import Frame, Box, Line
from compas.geometry import Scale
from compas.datastructures import Mesh
from compas.geometry import oriented_bounding_box_numpy
from compas_view2.app import App


# for face in final_block.faces():
#     if final_block.face_attribute(face,'top') == True:
#         id_top = face
#     if final_block.face_attribute(face,'bottom') == True:
#         id_bot = face

bottom = my_block.face_vertices(0)
top = my_block.face_vertices(1)[::-1]

viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

for edge in final_block.edges():
    if edge[0] == bottom[0] and edge[1] == top[0]:
        a, b = final_block.edge_coordinates(*edge)
        line = Line(a, b)
        viewer.add(line, linecolor=(0,1.0,0), linewidth=10)
        wire_edge = edge


viewer.add(final_block)
viewer.show()

<img style="float: right;" src="img/diagrams/7_edges.png" width="400">

## Step 7: Find the edges and extend them to the top face of the blank

This step makes sure that the wire path is properly considered by extending each path to the faces of the blank. 



In [7]:
import os
import compas

from compas.geometry import Frame, Box, Line, Point
from compas.datastructures import Mesh

from compas.geometry import intersection_line_plane, bestfit_plane_numpy
from compas_view2.app import App

# 1. use edge strip to find all edges on side of block
side_edges = final_block.edge_strip(wire_edge)

# 2. get vertices for top and bottom, make lists of the vertices in each
top_vertices = blank.top
bot_vertices = blank.bottom
all_vertices = blank.vertices

blank_top = []
for i in top_vertices:
    blank_top.append(all_vertices[i])
    
blank_bot = []
for i in bot_vertices:
    blank_bot.append(all_vertices[i])

# 3. generate a bestfit plane of the corners
plane_top = Plane(*bestfit_plane_numpy(blank_top))
plane_bot = Plane(*bestfit_plane_numpy(blank_bot))

wires = []
for edge in side_edges:
    a, b = final_block.edge_coordinates(*edge)
    line = Line(a, b)

    pt_a = Point(* intersection_line_plane(line, plane_top))
    pt_b = Point(* intersection_line_plane(line, plane_bot))
    wires.append((pt_a,pt_b))


viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10
viewer.add(final_block, opacity=0.65)
viewer.add(plane_top, opacity=0.4)
viewer.add(blank, opacity=0.3)

for a, b in wires:
    viewer.add(Line(a, b), linewidth=10, color=(1, 0, 0))

viewer.show()
    

<img style="float: right;" src="img/diagrams/8_interpolate.png" width="300">
<img style="float: right;" src="img/diagrams/8_edgePairs_labelled_final-01.png" width="300">

## Step 8: Pair the edges together and Interpolate

A wirecutter is one continuous wire with some width dimensions. This line must remain straight, however it is able to perform some rotations. Pairing the edges together will provide two rails for each side of the wirecutter to follow in order to cut the side faces.

Next, we evaluate the distance between edges and generate more edges for the machine to follow such that it will move at a consistend speed throughout the cutting process

In [8]:
import os
import compas

from compas.geometry import Frame, Box, Line
from compas.datastructures import Mesh

from compas.utilities import pairwise, linspace
from compas_view2.app import App

A, B = zip(*wires)

interpolation = []

# Zip together pairs of points on cycle A and cycle B.

for (a, aa), (b, bb) in zip(pairwise(A), pairwise(B)):

    # For each vector between pairs of points
    a_aa = aa - a
    b_bb = bb - b

    # Identify the length of the longest of the two
    # compute the approximate number of steps required to move 0.01 units at a time.

    l = max(a_aa.length, b_bb.length)
    n = int(l / 0.05)

    for i in linspace(0, 1, num=n):
        if i == 0:
            continue

        ai = a + a_aa * i
        bi = b + b_bb * i

        interpolation.append((ai, bi))
    

interpolation.reverse()

viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10
viewer.add(final_block)
viewer.add(blank, opacity=0.3)

# for a, b in wires:
#     viewer.add(Line(a, b), linewidth=10, color=(1, 0, 0))

@viewer.on(interval=10, frames=len(interpolation))
def cut(i):
    a, b = interpolation[i]
    viewer.add(Line(a, b), color=(1, 0, 0))

viewer.show()

<img style="float: right;" src="img/diagrams/9_top_cuts_labelled-01.png" width="400">

## Step 9: Find plane and intersections to cut the top

The face of the block which remains to be cut will not necessarily be flat. It is also not guaranteed to have a geometry which has two edges we can pair together and use as a path. 

Therefore, we instead find a bestfit plane for the top face, and extend that plane outwards to find where it intersects on the vertical edges of the blank material. 


In [9]:
import os
import compas

from compas.geometry import Frame, Box, Line, Point, Plane, Scale
from compas.geometry import bestfit_plane_numpy, intersection_plane_plane
from compas.datastructures import Mesh
from compas_view2.app import App

# 1. get a list of the bottom faces
bottom = list(final_block.faces_where({'bottom': True}))[0]

# 2. get corner vertex coordinates the top faces
corners = final_block.face_coordinates(bottom)
print(corners)

# 3. generate a bestfit plane of the corners
plane = Plane(*bestfit_plane_numpy(corners))

left_vertices = blank.left
right_vertices = blank.right
all_vertices = blank.vertices

blank_left = []
for i in left_vertices:
    blank_left.append(all_vertices[i])

# 3. generate a bestfit plane of the corners
left_plane = Plane(*bestfit_plane_numpy(blank_left))

blank_right = []
for i in right_vertices:
    blank_right.append(all_vertices[i])

# 3. generate a bestfit plane of the corners
right_plane = Plane(*bestfit_plane_numpy(blank_right))

inter = intersection_plane_plane(plane, left_plane)
int_line = Line(inter[0], inter[1])

inter_r = intersection_plane_plane(plane, right_plane)
int_line_r = Line(inter_r[0], inter_r[1])

front_vertices = blank.front
back_vertices = blank.back

blank_front = []
for i in front_vertices:
    blank_front.append(all_vertices[i])

blank_back = []
for i in back_vertices:
    blank_back.append(all_vertices[i])

front_plane = Plane(*bestfit_plane_numpy(blank_front))
back_plane = Plane(*bestfit_plane_numpy(blank_back))

top_wire_start = Point(* intersection_line_plane(int_line, front_plane))
top_wire_end = Point(* intersection_line_plane(int_line, back_plane))

top_wire_start2 = Point(* intersection_line_plane(int_line_r, front_plane))
top_wire_end2 = Point(* intersection_line_plane(int_line_r, back_plane))

top_wires = []
top_wires.append((top_wire_start,top_wire_end))
top_wires.append((top_wire_start2,top_wire_end2))

viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10
viewer.add(final_block)
viewer.add(plane, color=(0,0.2,0))

viewer.add(int_line, linewidth=10, color=(1, 0, 0))
viewer.add(int_line_r, linewidth=10, color=(0, 0, 1))

viewer.add(top_wire_start)
viewer.add(top_wire_end)
viewer.add(top_wire_start2)
viewer.add(top_wire_end2)
viewer.add(blank, opacity=0.4)
viewer.show()

[[0.3628682531636708, -0.6086869005184354, 2.3073226670246014], [-0.7590910682914109, -0.5554600788807651, 2.572987972916516], [-0.795879340426065, 0.19678956537118308, 2.4712812052755058], [0.01156574537728452, 0.7668080004832474, 2.303035844437432], [0.6605497034543009, 0.3094940963070285, 2.494494911107951], [0.9800841834927139, -0.1406465527017362, 2.613426935719804]]


<a id='wires'></a>
<img style="float: right;" src="img/diagrams/10_top.png" width="400">

## Step 10: Pair the Intersections and make the wirecutting path.
We are now able to use those intersections to create two pairs of edges, which then become the wirecutting path.

In [10]:
import os
import compas

from compas.geometry import Frame, Box, Line
from compas.datastructures import Mesh

from compas.utilities import pairwise, linspace
from compas_view2.app import App

A, B = zip(*top_wires)

interpolation = []

# Zip together pairs of points on cycle A and cycle B.

for (a, aa), (b, bb) in zip(pairwise(A), pairwise(B)):

    # For each vector between pairs of points
    a_aa = aa - a
    b_bb = bb - b

    # Identify the length of the longest of the two
    # compute the approximate number of steps required to move 0.01 units at a time.

    l = max(a_aa.length, b_bb.length)
    n = int(l / 0.05)

    for i in linspace(0, 1, num=n):
        if i == 0:
            continue

        ai = a + a_aa * i
        bi = b + b_bb * i

        interpolation.append((ai, bi))
    

interpolation.reverse()

viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10
viewer.add(final_block)
viewer.add(blank, opacity=0.3)

@viewer.on(interval=10, frames=len(interpolation))
def cut(i):
    a, b = interpolation[i]
    viewer.add(Line(a, b), color=(1, 0, 0))

viewer.show()